### Riddler Classic: 

In this challenge, the numbers from 1 to 11 are arranged in a circle in a particular order: 1, 4, 8, 7, 11, 2, 5, 9, 3, 6, 10. You then have to connect pairs of numbers with straight line segments that don’t intersect, and your score is the sum of the products of the joined numbers. For example, with the connections {1, 4}, {8, 10}, {3, 7}, {5, 9}, and {2, 11} (and the 6 left by itself), you get a score of 1·4 + 8·10 + 3·7 + 5·9 + 2·11, or 172.

The best score you can achieve with this ordering of 1 through 11 around the circle is 237, which you can get with the following connections: {6, 10}, {3, 4}, {7, 8}, {9, 11} and {2, 5} (and the 1 left by itself).

This got Friend-of-The-Riddler Tyler Barron and me thinking about possible extensions of this challenge. If you want the highest possible maximum score, then you can rearrange the numbers from 1 to 11 so that they are in numerical order around the circle. (With this arrangement, the maximum score is 250.)

But what if you want the lowest possible maximum score? That is, how can you order the numbers from 1 to 11 around the circle so that the maximum possible score is as low as possible? And what is the resulting score?

## Initial Thought:

- as we take a sequence, confirm that it does not cross paths with existing sequence:
    - we can read all of these circles as a sequential list 
    - in order to not cross elements must be:
        - sequence is within the other sequence
        - sequence is outside the other sequence 
        
### Determining An Intersection: 

In [1]:
# the list
list_order = [1, 4, 8, 7, 11, 2, 5, 9, 3, 6, 10]

# prior sequence:
seq_1 = (1,4)
seq_2 = (8,10)

#find remaining values:
remaining = [x for x in list_order if x not in seq_1 and x not in seq_2]
print(remaining)

[7, 11, 2, 5, 9, 3, 6]


In [2]:
def checkIntersection(newS, oldS, origList):
    """Check if there is intersection of newS on oldS"""
    # find index of elements in new seq
    new_a = origList.index(newS[0])
    new_b = origList.index(newS[1])
    
    # find index of elements in old seq
    old_a = origList.index(oldS[0])
    old_b = origList.index(oldS[1])
    
    # check if intersection or not 
    n_a, n_b = min(new_a,new_b), max(new_a,new_b)
    o_a, o_b = min(old_a,old_b), max(old_a,old_b)
    
    #print(n_a, n_b, o_a, o_b)
    
    if o_a < n_a < o_b and o_a < n_b < o_b:
        return 0

    # route is totally outside 
    if (not(o_a <= n_a <= o_b)) and (not(o_a <= n_b <= o_b)):
        return 0

    # intersect
    return 1

In [3]:
# do a few checks: 

#  1,10 and 4,6 should not intersect
assert(checkIntersection((1,10), (4,6), list_order) == 0)

#  1,9 and 4,3 should intersect
assert(checkIntersection((1,9), (3,4), list_order) == 1)

# 2,1 and 11,6 should intersect
assert(checkIntersection((2,1), (11,6), list_order) == 1)

# 10, 2 and 11,1 should not intersect
assert(checkIntersection((2,10), (11,1), list_order) == 0)

### Determine Eligible Sequences: 

- Now need a process for building eligible sequences

In [7]:
from math import floor, factorial
import random

# We have 11 elements, so will need 5 pairs 
sequence_count = floor(len(list_order)/2)
print(sequence_count)

5


In [8]:
factorial(5)

120

In [15]:
# within a run:
seq_dict = {}
current_list = list_order.copy()
for i in range(sequence_count):
    print(f"working in step {i}")
    current_seq = seq_dict.values()
    #print(current_list)
    tested_pairs = set()
    
    # possible pairs
    # add logic to determine if all unique pairs have been tested
    # just a combination, right? 
    # (n!) / k!(n-k!)
    pos_pairs = factorial(len(current_list)) / (factorial(2) * factorial(len(current_list) - 2))
    
    
    while True:
        
        # randomly find two numbers from list
        num1 = random.choices(current_list, k=1)[0]
        num2 = random.choices([x for x in current_list if x != num1], k=1)[0]
        
        # num1,num2 need to be added so we can see if we tested a pair
        tested_pairs.add((num1,num2))
        
        # check if tested pairs == unique pairs
        if len(tested_pairs) >= pos_pairs:
            print("Not possible to solve")
            break

        
        # compare to existing sequences - if intersection then rerun
        sum_c = 0
        for seq in current_seq:
            sum_c += checkIntersection((num1,num2), seq, list_order)
            
        # if > 0 then we had intersection and must redo, else store off
        if sum_c > 0:
            continue
        else:
            
            print(f"Found a clean pair!")
            
            # add tuple
            seq_dict[i] = (num1,num2)
            
            # remove from current list 
            current_list.remove(num1)
            current_list.remove(num2)
            
            break
            

working in step 0
Found a clean pair!
working in step 1
Found a clean pair!
working in step 2
Found a clean pair!
working in step 3
Found a clean pair!
working in step 4
Found a clean pair!


In [16]:
seq_dict

{0: (6, 10), 1: (8, 1), 2: (3, 7), 3: (11, 2), 4: (5, 9)}

### Iterate N Times

Run a big simulation, keeping tabs on the max sequence. Can we match what the riddler had? 

`The best score you can achieve with this ordering of 1 through 11 around the circle is 237, which you can get with the following connections: {6, 10}, {3, 4}, {7, 8}, {9, 11} and {2, 5} (and the 1 left by itself).`

Able to solve!

In [40]:
import time 

n = 5_000

max_seq = {}
max_score = 0

start = time.time()

for _ in range(n):
    # within a run:
    seq_dict = {}
    current_list = list_order.copy()
    for i in range(sequence_count):
        #print(f"working in step {i}")
        current_seq = seq_dict.values()
        #print(current_list)
        tested_pairs = set()

        # possible pairs
        # add logic to determine if all unique pairs have been tested
        # just a combination, right? 
        # (n!) / k!(n-k!)
        pos_pairs = factorial(len(current_list)) / (factorial(2) * factorial(len(current_list) - 2))


        while True:

            # randomly find two numbers from list
            num1 = random.choices(current_list, k=1)[0]
            num2 = random.choices([x for x in current_list if x != num1], k=1)[0]

            # num1,num2 need to be added so we can see if we tested a pair
            tested_pairs.add((num1,num2))

            # check if tested pairs == unique pairs
            if len(tested_pairs) >= pos_pairs:
                #print("Not possible to solve")
                break


            # compare to existing sequences - if intersection then rerun
            sum_c = 0
            for seq in current_seq:
                sum_c += checkIntersection((num1,num2), seq, list_order)

            # if > 0 then we had intersection and must redo, else store off
            if sum_c > 0:
                continue
            else:

                #print(f"Found a clean pair!")

                # add tuple
                seq_dict[i] = (num1,num2)
                
                

                # remove from current list 
                current_list.remove(num1)
                current_list.remove(num2)
                
                # # if 5 total, then calculate 
                if len(seq_dict) == 5:
                    score = 0
                    for key, val in seq_dict.items():
                        score += val[0] * val[1]

                # determine if exceeds max, if so save off 
                
                if score > max_score:
                    max_score = score
                    max_seq = seq_dict
                    
                break
finish = time.time()

print(finish - start)
print(f"New max score: {max_score}")

0.5435709953308105
New max score: 237


In [41]:
max_seq

{0: (9, 11), 1: (10, 6), 2: (3, 4), 3: (2, 5), 4: (7, 8)}

In [44]:
# time to process all combinations? 251 days...
factorial(11) * (finish - start) * (1/ 86400)

251.12979984283447

### Final Step: 

- Need to do this for all orderings of the numbers, which is actually a massive problem. It would be `11!`


One way to fix this is use a generator to build next sequence, once a max score exceed prior min we move to next ordering.

May need to switch to numpy.

In [33]:
factorial(11)

39916800